# Memory
Memory 是让 Agent 记住先前交互信息的系统，分为 短时记忆（Short-term Memory） 和 长时记忆（Long-term Memory） 两种。 它让 Agent 能记住用户偏好、对话历史，实现个性化交互。

LangGraph 支持两种对于构建对话代理至关重要的内存类型：

- 短期内存：通过在会话中维护消息历史来跟踪正在进行的对话。
- 长期内存：在不同会话之间存储用户特定或应用程序级别的数据。

|特性|	Checkpointer|	Store|
|----|----|----|
|范围	|单线程完整状态	|跨线程 key-value|
|自动保存	|每个 super-step	|手动调用|
|用途	|对话历史、时间旅行	|用户偏好、配置|
|访问	|thread_id	|namespace + key|


<img src="./assets/memory.png" align="left" width="700">

## 初始化
加载并检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")

DASHSCOPE_API_KEY=****931f
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****ef8f
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****s-56
LANGSMITH_ENDPOINT=****.com


In [2]:
#导入实例数据库
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

#定义运行时上下文，为Agent和tools提供数据库访问。
from dataclasses import dataclass

# 定义上下文结构以支持依赖注入
@dataclass
class RuntimeContext:
    """运行时上下文，保存数据库连接供Agent和tools使用。"""
    db: SQLDatabase

from langchain.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """执行SQL查询并返回结果。"""
    runtime = get_runtime(RuntimeContext)   # 取出运行时上下文
    db = runtime.context.db                 # 获取数据库连接
    try:
        return db.run(query)                # 进行数据库查询
    except Exception as e:
        return f"执行查询时出错: {e}"
    

SYSTEM_PROMPT = """你是一名严谨的 SQLite 分析师。

规则：
- 逐步思考。
- 如需数据,请使用名为“execute_sql”的工具并仅执行一个 SELECT 查询。
- 仅读取数据，禁止执行 INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE 操作。
- 输出结果最多限制为 5 行，除非用户明确另有要求。
- 如果工具返回“执行查询时出错:”,请修改 SQL 语句并再次尝试。
- 优先明确列出列名；避免使用 SELECT * 。
"""


from langchain_qwq import ChatQwen
import os
model=ChatQwen(
    model="qwen3-max", 
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    )
from langchain.agents import create_agent

agent=create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

In [3]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================

I'll help you find the total on your last invoice, Frank. First, let me check if I can find your customer record in the database.
Tool Calls:
  execute_sql (call_964cc366eeb2415cbeb6787c)
 Call ID: call_964cc366eeb2415cbeb6787c
  Args:
    query: SELECT customer_id FROM customers WHERE name = 'Frank Harris';
================================= Tool Message =================================
Name: execute_sql

执行查询时出错: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT customer_id FROM customers WHERE name = 'Frank Harris';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================

I apologize for the error. It seems the database structure is different than I expected. Le

In [4]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

To provide you with the titles, I need to know which specific table or context you are referring to. Could you please specify the table name or provide more details?


## 添加记忆

In [5]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

question = "我是弗兰克·哈里斯，我上一张发票的总金额是多少?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

我是弗兰克·哈里斯，我上一张发票的总金额是多少?
================================== Ai Message ==================================

为了找到弗兰克·哈里斯上一张发票的总金额，我需要先确认数据库中的相关表结构。通常，客户信息和发票信息会分别存储在不同的表中，例如 `customers` 和 `invoices` 表。

首先，我会检查是否有包含客户姓名和发票信息的表。
Tool Calls:
  execute_sql (call_51718738e2a84ecda9363e3b)
 Call ID: call_51718738e2a84ecda9363e3b
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================

数据库中存在 `Customer` 和 `Invoice` 表，这很适合用来查询弗兰克·哈里斯的发票信息。接下来，我将检查 `Customer` 表的结构，以确认如何找到弗兰克·哈里斯的客户记录。
Tool Calls:
  execute_sql (call_05a49f5ead7943c09874512b)
 Call ID: call_05a49f5

In [6]:
question = "有哪些标题?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

有哪些标题?
================================== Ai Message ==================================

为了明确您的问题，我假设您是在询问数据库中某个表的“标题”（可能是列名或某种特定字段）。由于“标题”一词可能有多种含义，我会先检查所有表的结构，看看是否有包含类似“标题”（如 `Title`）字段的表。

首先，我将查找哪些表包含名为 `Title` 的列。
Tool Calls:
  execute_sql (call_25465058998049a1a27c9d1a)
 Call ID: call_25465058998049a1a27c9d1a
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' AND sql LIKE '%Title%';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Employee',)]
================================== Ai Message ==================================

在 `Album` 和 `Employee` 表中可能存在名为 `Title` 的字段。接下来，我将分别检查这两个表的结构以确认。
Tool Calls:
  execute_sql (call_95cabe1f80fd4d83882069ba)
 Call ID: call_95cabe1f80fd4d83882069ba
  Args:
    query: PRAGMA table_info(Album);
================================= Tool Message =================================
Name: ex

在这个例子中，在添加 checkpointer 后，第一次调用时模型会进行 tool call 进行 sql 查询，第二次调用时，模型有了记忆，便直接进行回复